<a href="https://colab.research.google.com/github/matsunagalab/tutorial_analyzingMDdata/blob/main/02_mdtraj_atomselection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MDTraj atom selection

## MDTrajのインストール

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:15
🔁 Restarting kernel...


In [1]:
!conda install -c conda-forge mdtraj

Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



## T4 LysozymeのMDデータの取得

`wildtype.pdb` と `wildtype.dcd` をダウンロードする。

In [2]:
!gdown 1Lu22Z7ARKSws77sBaYr84iCcQsnTHId9
!gdown 1Eh0SSSxgAmv7LI2NvAY43tLMLI63lSa-

Downloading...
From: https://drive.google.com/uc?id=1Lu22Z7ARKSws77sBaYr84iCcQsnTHId9
To: /content/wildtype.pdb
100% 214k/214k [00:00<00:00, 77.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1Eh0SSSxgAmv7LI2NvAY43tLMLI63lSa-
From (redirected): https://drive.google.com/uc?id=1Eh0SSSxgAmv7LI2NvAY43tLMLI63lSa-&confirm=t&uuid=56fdff72-6469-4049-b4a0-d93bfa10a6c6
To: /content/wildtype.dcd
100% 318M/318M [00:03<00:00, 104MB/s]


## MDTrajでの原子選択

以下の文書は主に MDTrajのドキュメント https://www.mdtraj.org/1.9.5/examples/atom-selection.html を日本語訳したものです。

MDTrajでの原子と残基の選択の基本について説明します。まず、例のトラジェクトリをロードします。

In [3]:
import mdtraj as md
import numpy as np

# DCDファイルからトラジェクトリを読み込み、対応するPDBトポロジーファイルを使用する
# トラジェクトリオブジェクト traj が作成される
traj = md.load('/content/wildtype.dcd', top='/content/wildtype.pdb')

# traj の中身を出力する
print(traj)

<mdtraj.Trajectory with 10000 frames, 2643 atoms, 164 residues, and unitcells>


traj.n_atoms や traj.n_residues を使用して、より直接的に原子数や残基数を確認することもできます。

In [4]:
print('原子は何個？ %s' % traj.n_atoms)
print('残基は何個？ %s' % traj.n_residues)

原子は何個？ 2643
残基は何個？ 164


また、traj.xyz を操作することで原子の位置を操作することもできます。これは、各原子のxyz座標を含むNumPy配列で、次元は(n_frames, n_atoms, 3)です。5フレーム目の10番目の原子の3D座標を探してみましょう。

In [5]:
frame_idx = 4 # ゼロから始まるフレーム番号
atom_idx = 9 # ゼロから始まる原子インデックス
print('10フレーム目の5番目の原子はどこにある？')
print('x: %s\ty: %s\tz: %s' % tuple(traj.xyz[frame_idx, atom_idx,:]))

10フレーム目の5番目の原子はどこにある？
x: 5.22431	y: 0.44880342	z: 0.6664297


トポロジーオブジェクト、すべてのTrajectoryオブジェクトにはTopologyが含まれています。TrajectoryのTopologyには、系（分子）の結合情報や特定のChain, Residue(残基), Atom情報が含まれています。

In [6]:
topology = traj.topology
print(topology)

<mdtraj.Topology with 1 chains, 164 residues, 2643 atoms, 2663 bonds>


トポロジーオブジェクトを使用すれば、特定の原子を選択したり、すべての原子をループしたりすることができます。 (注意: zero-based indexです)

In [7]:
print('5番目の原子: %s' % topology.atom(4))
print('すべての原子: %s' % [atom for atom in topology.atoms])

5番目の原子: MET1-CA
すべての原子: [MET1-N, MET1-H, MET1-H2, MET1-H3, MET1-CA, MET1-HA, MET1-C, MET1-O, MET1-CB, MET1-HB2, MET1-HB3, MET1-CG, MET1-HG2, MET1-HG3, MET1-SD, MET1-CE, MET1-HE1, MET1-HE2, MET1-HE3, ASN2-N, ASN2-H, ASN2-CA, ASN2-HA, ASN2-C, ASN2-O, ASN2-CB, ASN2-HB2, ASN2-HB3, ASN2-CG, ASN2-OD1, ASN2-ND2, ASN2-HD21, ASN2-HD22, ILE3-N, ILE3-H, ILE3-CA, ILE3-HA, ILE3-C, ILE3-O, ILE3-CB, ILE3-HB, ILE3-CG1, ILE3-HG12, ILE3-HG13, ILE3-CG2, ILE3-HG21, ILE3-HG22, ILE3-HG23, ILE3-CD1, ILE3-HD11, ILE3-HD12, ILE3-HD13, PHE4-N, PHE4-H, PHE4-CA, PHE4-HA, PHE4-C, PHE4-O, PHE4-CB, PHE4-HB2, PHE4-HB3, PHE4-CG, PHE4-CD1, PHE4-HD1, PHE4-CD2, PHE4-HD2, PHE4-CE1, PHE4-HE1, PHE4-CE2, PHE4-HE2, PHE4-CZ, PHE4-HZ, GLU5-N, GLU5-H, GLU5-CA, GLU5-HA, GLU5-C, GLU5-O, GLU5-CB, GLU5-HB2, GLU5-HB3, GLU5-CG, GLU5-HG2, GLU5-HG3, GLU5-CD, GLU5-OE1, GLU5-OE2, MET6-N, MET6-H, MET6-CA, MET6-HA, MET6-C, MET6-O, MET6-CB, MET6-HB2, MET6-HB3, MET6-CG, MET6-HG2, MET6-HG3, MET6-SD, MET6-CE, MET6-HE1, MET6-HE2, MET6-HE3, LEU7-N

Residue(残基)についても同じことが言えます。

In [8]:
print('2番目の残基: %s' % traj.topology.residue(1))
print('すべての残基: %s' % [residue for residue in traj.topology.residues])

2番目の残基: ASN2
すべての残基: [MET1, ASN2, ILE3, PHE4, GLU5, MET6, LEU7, ARG8, ILE9, ASP10, GLU11, GLY12, LEU13, ARG14, LEU15, LYS16, ILE17, TYR18, LYS19, ASP20, THR21, GLU22, GLY23, TYR24, TYR25, THR26, ILE27, GLY28, ILE29, GLY30, HIS31, LEU32, LEU33, THR34, LYS35, SER36, PRO37, SER38, LEU39, ASN40, ALA41, ALA42, LYS43, SER44, GLU45, LEU46, ASP47, LYS48, ALA49, ILE50, GLY51, ARG52, ASN53, CYS54, ASN55, GLY56, VAL57, ILE58, THR59, LYS60, ASP61, GLU62, ALA63, GLU64, LYS65, LEU66, PHE67, ASN68, GLN69, ASP70, VAL71, ASP72, ALA73, ALA74, VAL75, ARG76, GLY77, ILE78, LEU79, ARG80, ASN81, ALA82, LYS83, LEU84, LYS85, PRO86, VAL87, TYR88, ASP89, SER90, LEU91, ASP92, ALA93, VAL94, ARG95, ARG96, CYS97, ALA98, LEU99, ILE100, ASN101, MET102, VAL103, PHE104, GLN105, MET106, GLY107, GLU108, THR109, GLY110, VAL111, ALA112, GLY113, PHE114, THR115, ASN116, SER117, LEU118, ARG119, MET120, LEU121, GLN122, GLN123, LYS124, ARG125, TRP126, ASP127, GLU128, ALA129, ALA130, VAL131, ASN132, LEU133, ALA134, LYS135, SER136

さらに、すべての原子と残基もオブジェクトであり、それぞれが独自のpropertyを持っています。以下は、そのうちのいくつかを説明するシンプルな例です。

In [9]:
atom = topology.atom(10)
print('''こんにちは！私は %s 番目の原子で、名前は %s です。
私は %s 原子で、%s 個の結合を持っています。
私は %s 残基の一部です。''' % (atom.index, atom.name, atom.element.name, atom.n_bonds, atom.residue.name))

こんにちは！私は 10 番目の原子で、名前は HB3 です。
私は hydrogen 原子で、1 個の結合を持っています。
私は MET 残基の一部です。


また、atom.is_sidechainやresidue.is_proteinのようなより複雑なプロパティもあります。これらを利用することで、よりパワフルな選択が可能になります。

これらのプロパティをPythonのフィルタリスト機能と組み合わせると強力です。たとえば、分子の側鎖にあるすべての Carbon atom のインデックスを求めたいとしましょう。こんな感じで実現できます。

In [ ]:
print([atom.index for atom in topology.atoms if atom.element.symbol == 'C' and atom.is_sidechain])

[6, 9, 13, 23, 26, 37, 39, 43, 46, 56, 59, 60, 62, 64, 66, 68, 76, 79, 82, 91, 94, 98, 108, 111, 113, 117, 127, 130, 133, 138, 151, 153, 157, 160, 170, 173, 182, 185, 188, 204, 207, 209, 213, 223, 226, 229, 234, 247, 250, 252, 256, 266, 269, 272, 275, 288, 290, 294, 297, 307, 310, 311, 313, 315, 318, 320, 328, 331, 334, 337, 350, 353, 362, 366, 376, 379, 382, 398, 401, 402, 404, 406, 409, 411, 419, 422, 423, 425, 427, 430, 432, 440, 444, 454, 456, 460, 463, 480, 482, 486, 489, 506, 511, 512, 515, 523, 526, 528, 532, 542, 545, 547, 551, 561, 565, 575, 578, 581, 584, 597, 605, 610, 613, 622, 633, 636, 638, 642, 652, 655, 666, 676, 686, 689, 692, 695, 708, 719, 722, 725, 734, 737, 739, 743, 753, 756, 765, 768, 771, 774, 787, 797, 799, 803, 806, 823, 826, 829, 834, 847, 850, 861, 872, 875, 893, 895, 899, 909, 911, 915, 918, 928, 932, 942, 945, 948, 951, 964, 967, 976, 979, 982, 991, 1001, 1004, 1007, 1016, 1019, 1022, 1025, 1038, 1041, 1043, 1047, 1057, 1060, 1061, 1063, 1065, 1067, 1069, 

または、最初の Chain のすべての 偶数インデックスの Residue (残基) を求めることもできます（ただし、この例では1つの鎖しかありません）

In [ ]:
print([residue for residue in topology.chain(0).residues if residue.index % 2 == 0])

[MET1, ILE3, GLU5, LEU7, ILE9, GLU11, LEU13, LEU15, ILE17, LYS19, THR21, GLY23, TYR25, ILE27, ILE29, HIS31, LEU33, LYS35, PRO37, LEU39, ALA41, LYS43, GLU45, ASP47, ALA49, GLY51, ASN53, ASN55, VAL57, THR59, ASP61, ALA63, LYS65, PHE67, GLN69, VAL71, ALA73, VAL75, GLY77, LEU79, ASN81, LYS83, LYS85, VAL87, ASP89, LEU91, ALA93, ARG95, CYS97, LEU99, ASN101, VAL103, GLN105, GLY107, THR109, VAL111, GLY113, THR115, SER117, ARG119, LEU121, GLN123, ARG125, ASP127, ALA129, VAL131, LEU133, LYS135, ARG137, TYR139, GLN141, PRO143, ARG145, LYS147, VAL149, THR151, PHE153, THR155, THR157, ASP159, TYR161, ASN163]


上記のようなフィルタリストのプログラミングに迷っている場合、MDTrajはPyMolやVMDに似た豊富な原子選択言語も提供しています。これには、topology.selectを使用してアクセスできます。最後の2つの残基のすべての原子を探してみましょう。

原子選択構文の詳細については、[ドキュメント](https://www.mdtraj.org/1.9.7/atom_selection.html)をご覧ください。

In [12]:
print(topology.select('resid 1 to 2'))

[19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42
 43 44 45 46 47 48 49 50 51]


より複雑な選択も可能です。以下では、主鎖のすべての Nitrogen atom を選択します。

In [13]:
print(topology.select('name N and backbone'))

[   0   19   33   52   72   87  104  123  147  166  178  193  200  219
  243  262  284  303  324  346  358  372  387  394  415  436  450  469
  476  495  502  519  538  557  571  593  604  618  629  648  662  672
  682  704  715  730  749  761  783  793  812  819  843  857  868  882
  889  905  924  938  960  972  987  997 1012 1034 1053 1073 1087 1104
 1116 1132 1144 1154 1164 1180 1204 1211 1230 1249 1273 1287 1297 1319
 1338 1360 1374 1390 1411 1423 1434 1453 1465 1475 1491 1515 1539 1550
 1560 1579 1598 1612 1629 1645 1665 1682 1699 1706 1721 1735 1742 1758
 1768 1775 1795 1809 1823 1834 1853 1877 1894 1913 1930 1947 1969 1993
 2017 2029 2044 2054 2064 2080 2094 2113 2123 2145 2156 2180 2204 2225
 2239 2256 2270 2284 2298 2322 2332 2354 2378 2394 2413 2427 2441 2461
 2485 2499 2506 2520 2544 2556 2566 2587 2609 2623]


これらの結果を生成するコードを見たい場合は、select_expressionを使用できます。これは、atom selection コードの文字列表現を返します。

In [14]:
selection = topology.select_expression('name CA and resid 1 to 2')
print(selection)

[atom.index for atom in topology.atoms if ((atom.name == 'CA') and (1 <= atom.residue.index <= 2))
]
